## Prerequisites

### Installing and Loading Packages

In [1]:
!pip install git+https://github.com/toon-format/toon-python.git

  Cloning https://github.com/toon-format/toon-python.git to /tmp/pip-req-build-z5x0vghs
  Running command git clone --filter=blob:none --quiet https://github.com/toon-format/toon-python.git /tmp/pip-req-build-z5x0vghs
  Resolved https://github.com/toon-format/toon-python.git to commit 9c4f0c0c24f2a0b0b376315f4b8707f8c9006de6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for toon_format: filename=toon_format-0.9.0b1-py3-none-any.whl size=36285 sha256=298ea641dec9bbf68d69226a6cd31b4cb504fe79b43bc82a1675c5be86b814bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-gd4_7j7a/wheels/71/5b/77/bc062fea7c190909f65ce491c147046ee0face81c520fba311
Successfully built toon_format


In [2]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 20.5 MB/s eta 0:00:00


In [3]:
!pip install transformers==4.46.3 jsonformer==0.12.0 tokenizers==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.2.0
    Uninstalling termcolor-3.2.0:
      Successfully uninstalled termcolor-3.2.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [4]:
import pandas as pd
import random
from datasets import load_dataset
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from huggingface_hub import login
from jsonformer import Jsonformer
import time
from toon_format import encode, decode

from mistralai import Mistral
import os

import warnings
warnings.filterwarnings('ignore')

import re
import torch
import sqlite3

torch.manual_seed(42)
random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)

Using device: cuda


In [5]:
mistral_key = "pK73M6WZZa1a0vxqKK00o2ERf8aEnkQL"

### Process Metadata

In [6]:
metadata = {
  "tables": [
    {
        "table_name": "customers",
        "columns":{
    "customer_id": "Unique identifier for each customer, format 'CUST_' followed by 6-digit number (string)",
    "name": "Full name of the customer (string)",
    "phone": "Phone number of the customer (string)",
    "city": "City where the customer resides (string, categorical examples: ['Minneapolis', 'New York', 'Chicago', 'Dallas', 'San Jose', 'Miami', 'Seattle'])",
    "state": "State where the customer resides (string, categorical examples: ['Minnesota', 'New York', 'Illinois', 'Texas', 'California', 'Florida', 'Washington'])",
    "zipcode": "ZIP code of the customer's location (string, categorical examples: ['55414', '10001', '60601', '75201', '95101', '33101', '98101'])",
    "age": "Age of the customer in years (integer, 20–75)",
    "gender": "Gender of the customer, categorical, values: ['male', 'female', 'nonbinary']",
    "marital_status": "Marital status, categorical, values: ['single', 'married', 'divorced', 'widowed']",
    "dependents": "Number of dependents (integer)",
    "education_level": "Highest education level, categorical, values: ['high_school', 'bachelor', 'master', 'phd']",
    "occupation": "Occupation of the customer (string, categorical, examples depend on age group)",
    "employment_status": "Employment status, categorical, values: ['employed', 'self-employed', 'unemployed', 'retired', 'student']",
    "annual_income": "Annual income in USD (integer)",
    "credit_score": "Credit score of the customer (integer, 550–850)",
    "existing_loans": "Type of existing loans, categorical, values: ['none', 'home', 'car', 'personal', 'multiple']",
    "avg_monthly_expense": "Average monthly expense in USD (float)",
    "vehicle_owner": "Whether the customer owns a vehicle (boolean, True/False)",
    "vehicle_type": "Type of vehicle owned, categorical, values: ['car', 'bike', 'none']",
    "budget_per_month": "Monthly budget allocated for insurance/expenses in USD (integer)",
    "customer_tenure_years": "Customer tenure in years (float, 0.5–10)",
    "coverage_preferences": "Preferred types of insurance coverage, list of 1–2 items, categorical, values: ['health', 'life', 'auto', 'home']",
    "risk_tolerance": "Risk tolerance level, categorical, values: ['Low', 'Medium', 'High']",
    "interest_tags": "List of interest tags related to lifestyle or insurance needs, examples: ['family', 'education', 'vehicle', 'retirement', 'travel', 'maternity']"
          }
    },
    {
        "table_name": "policies",
        "columns": {
    "policy_id": "Unique identifier for each policy, format 'POL_' followed by 6-digit number (string)",
    "customer_id": "Identifier for the customer who owns the policy (string, foreign key to customers table)",
    "policy_type": "Type of insurance policy, categorical, values: ['health', 'life', 'auto', 'home']",
    "premium_amount": "Premium amount charged for the policy in USD (integer)",
    "coverage_amount": "Coverage amount provided by the policy in USD (integer)",
    "policy_duration_years": "Duration of the policy in years (integer)",
    "policy_start_year": "Start year of the policy (integer, e.g., 2015–2024)",
    "policy_end_year": "End year of the policy (integer)",
    "policy_risk_level": "Risk level assigned to the policy, categorical, values: ['low', 'medium', 'high']",
    "policy_status": "Current status of the policy, categorical, values: ['active', 'inactive']"
        }
    },
    {
        "table_name": "submissions",
        "columns":{
    "submission_id": "Unique identifier for each submission, format 'SUB-' followed by 6-character alphanumeric code (string)",
    "submission_date": "Date when the submission was created (YYYY-MM-DD, string)",
    "policy_type": "Type of insurance policy, categorical, values: ['health', 'life', 'auto', 'home']",
    "statusid": "Submission status code, categorical, values: ['BND' (Bound), 'QTE' (Quote Only), 'DEC' (Declined), 'UNQ' (Unquoted)]",
    "assignment_date": "Date when submission was assigned to an agent (YYYY-MM-DD, string, may be empty)",
    "first_contact": "Date of first contact with customer (YYYY-MM-DD, string, may be empty)",
    "quote_date": "Date when quote was provided (YYYY-MM-DD, string, may be empty)",
    "declined_date": "Date when submission was declined (YYYY-MM-DD, string, may be empty)",
    "contacted": "Indicates if customer was contacted, categorical, values: ['Yes', 'No']",
    "overwritten_flag": "Indicates if submission was overwritten, categorical, values: ['Yes', 'No']",
    "reclassification_flag": "Indicates if submission was reclassified, categorical, values: ['Yes', 'No']",
    "additional_info_requested": "Indicates if additional info was requested, categorical, values: ['Yes', 'No']",
    "agent_id": "Identifier for the agent assigned to this submission (string, foreign key to agents table)",
    "policy_id": "Identifier for the policy bound to this submission, if applicable (string, foreign key to policies table, may be empty)"
}
    },

    {
       "table_name": "claims",
       "columns":{
    "claim_id": "Unique identifier for each claim, format 'CLM_' followed by 6-digit number (string)",
    "policy_id": "Identifier for the policy associated with the claim (string, foreign key to policies table)",
    "customer_id": "Identifier for the customer making the claim (string, foreign key to customers table)",
    "claim_date": "Date when the claim was made (YYYY-MM-DD, string)",
    "claim_amount": "Total claimed amount in USD (float)",
    "approved_amount": "Amount approved for payout; 0 if rejected (float)",
    "claim_status": "Status of the claim, categorical, values: ['approved', 'rejected']",
    "processing_time_days": "Number of days taken to process the claim (integer)",
    "settlement_date": "Date when the claim was settled; None if rejected (YYYY-MM-DD, string or null)",
    "fraud_flag": "Indicates potential fraud, binary, values: [0, 1]"
}
    },

  {
    "table_name": "agents",
    "columns":{
    "agent_id": "Unique identifier for each agent, format 'A' followed by a number (e.g., A1000), categorical",
    "agent_name": "Full name of the agent (string)",
    "agent_license_number": "Unique license number for the agent, format 'LIC-' followed by 8-character alphanumeric code (string)",
    "agency_name": "Name of the agency the agent belongs to (string)",
    "channel": "The sales channel the agent operates in, categorical, values: ['Telemarketing', 'Digital', 'Branch']",
    "policy_type": "Line of business the agent handles, categorical, values: ['auto', 'home', 'health', 'life']"
    }
  }
]
}

In [7]:
# metadata = {
#   "tables": [
#     {
#       "table_name": "customers",
#       "description": "Stores customer demographic, financial, and preference information.",
#       "columns": {
#         "customer_id": "Unique identifier for each customer.",
#         "name": "Full name of the customer.",
#         "phone": "Customer phone number.",
#         "city": "City of residence.",
#         "state": "State of residence.",
#         "zipcode": "Zip code of the customer's address.",
#         "age": "Age of the customer.",
#         "gender": "Gender of the customer.",
#         "marital_status": "Marital status (single, married, etc).",
#         "dependents": "Number of dependents the customer supports.",
#         "education_level": "Highest education level attained.",
#         "occupation": "Customer's job role or profession.",
#         "employment_status": "Employment type (full-time, unemployed, etc).",
#         "annual_income": "Yearly income reported by the customer.",
#         "credit_score": "Customer’s creditworthiness score.",
#         "existing_loans": "Count of loans currently held by the customer.",
#         "avg_monthly_expense": "Customer's average monthly spending.",
#         "vehicle_owner": "Whether customer owns a vehicle (yes/no).",
#         "vehicle_type": "Type of vehicle owned, if any.",
#         "budget_per_month": "Monthly budget for insurance.",
#         "customer_tenure_years": "Years the customer has been with the company.",
#         "coverage_preferences": "Preferred types of coverage (health, auto, etc).",
#         "risk_tolerance": "Customer’s tolerance for risk (low/medium/high).",
#         "interest_tags": "Customer interest indicators used for marketing."
#       }
#     },
#     {
#       "table_name": "policies",
#       "description": "Contains details of issued insurance policies.",
#       "columns": {
#         "policy_id": "Unique identifier for each policy.",
#         "customer_id": "Foreign key referencing customer owning the policy.",
#         "policy_type": "Type of insurance policy (auto, health, home, etc).",
#         "provider": "Insurance provider issuing the policy.",
#         "premium_amount": "Amount paid periodically for the policy.",
#         "coverage_amount": "Maximum coverage amount or benefit limit.",
#         "policy_duration_years": "Total duration of the policy in years.",
#         "policy_start_year": "Year the policy started.",
#         "policy_end_year": "Year the policy ends or expires.",
#         "policy_risk_level": "Underwriting-assigned risk level of policy.",
#         "policy_status": "Current status (active, expired, cancelled).",
#         "customer_income": "Income of the customer at policy issuance.",
#         "customer_credit_score": "Credit score used for underwriting.",
#         "customer_employment": "Customer employment info related to underwriting.",
#         "rationale": "Reasoning for underwriting decisions."
#       }
#     },
#     {
#       "table_name": "submissions",
#       "description": "Represents incoming insurance leads and quote processing.",
#       "columns": {
#         "submission_id": "Unique identifier for each submission.",
#         "submission_date": "Date when submission was received.",
#         "lob": "Line of business (auto, health, etc).",
#         "status_id": "Status code of submission workflow.",
#         "assignment_date": "Date assigned to an agent.",
#         "first_contact": "Timestamp of first customer interaction.",
#         "quote_date": "Date when a quote was provided.",
#         "declined_date": "Date submission was declined.",
#         "contacted": "Indicates whether customer was contacted.",
#         "overwritten_flag": "Flag indicating if submission was overwritten.",
#         "reclassification_flag": "Flag if submission was reclassified.",
#         "additional_info_requested": "If more documents/info were required.",
#         "agent_id": "Foreign key referencing agent handling the submission.",
#         "policy_id": "Foreign key linking to policy if converted."
#       }
#     },
#     {
#       "table_name": "agents",
#       "description": "Stores information about agents selling or managing policies.",
#       "columns": {
#         "agent_id": "Unique identifier for each agent.",
#         "agent_name": "Name of the agent.",
#         "agent_license_number": "Official license or certification number.",
#         "agency_name": "Name of agency the agent belongs to.",
#         "channel": "Distribution channel (online, broker, direct).",
#         "policy_type": "Type of policies the agent specializes in."
#       }
#     },
#     {
#       "table_name": "claims",
#       "description": "Tracks claims filed against insurance policies.",
#       "columns": {
#         "claim_id": "Unique identifier for each claim.",
#         "policy_id": "Foreign key referencing the policy being claimed.",
#         "customer_id": "Foreign key referencing customer owning the policy."
#         "claim_date": "Date when the claim was filed.",
#         "claim_amount": "Total amount requested by claimant.",
#         "approved_amount": "Amount approved for payout.",
#         "claim_status": "Current claim status (approved, rejected, etc).",
#         "settlement_date": "Date when claim was resolved.",
#         "processing_time_days": "Time taken to process claim.",
#         "fraud_flag": "Indicates suspected fraudulent claim."
#       }
#     }
#   ]
# }

In [8]:
new_metadata = dict()
for i in metadata['tables']:
  new_metadata[i['table_name']] = dict()
  # new_metadata[i['table_name']]['description'] = i['description']
  new_metadata[i['table_name']]['columns'] = i['columns']

In [9]:
new_metadata

{'customers': {'columns': {'customer_id': "Unique identifier for each customer, format 'CUST_' followed by 6-digit number (string)",
   'name': 'Full name of the customer (string)',
   'phone': 'Phone number of the customer (string)',
   'city': "City where the customer resides (string, categorical examples: ['Minneapolis', 'New York', 'Chicago', 'Dallas', 'San Jose', 'Miami', 'Seattle'])",
   'state': "State where the customer resides (string, categorical examples: ['Minnesota', 'New York', 'Illinois', 'Texas', 'California', 'Florida', 'Washington'])",
   'zipcode': "ZIP code of the customer's location (string, categorical examples: ['55414', '10001', '60601', '75201', '95101', '33101', '98101'])",
   'age': 'Age of the customer in years (integer, 20–75)',
   'gender': "Gender of the customer, categorical, values: ['male', 'female', 'nonbinary']",
   'marital_status': "Marital status, categorical, values: ['single', 'married', 'divorced', 'widowed']",
   'dependents': 'Number of depen

### Getting these in a toon-format

In [10]:
metadata_toon = ''
for i in new_metadata:
  metadata_toon += f'Table: {i}\n'
  metadata_toon += encode(new_metadata[i])
  metadata_toon += '\n\n'

In [11]:
print(metadata_toon)

Table: customers
columns:
  customer_id: "Unique identifier for each customer, format 'CUST_' followed by 6-digit number (string)"
  name: Full name of the customer (string)
  phone: Phone number of the customer (string)
  city: "City where the customer resides (string, categorical examples: ['Minneapolis', 'New York', 'Chicago', 'Dallas', 'San Jose', 'Miami', 'Seattle'])"
  state: "State where the customer resides (string, categorical examples: ['Minnesota', 'New York', 'Illinois', 'Texas', 'California', 'Florida', 'Washington'])"
  zipcode: "ZIP code of the customer's location (string, categorical examples: ['55414', '10001', '60601', '75201', '95101', '33101', '98101'])"
  age: "Age of the customer in years (integer, 20–75)"
  gender: "Gender of the customer, categorical, values: ['male', 'female', 'nonbinary']"
  marital_status: "Marital status, categorical, values: ['single', 'married', 'divorced', 'widowed']"
  dependents: Number of dependents (integer)
  education_level: "Highes

In [12]:
# Fetching KPIs in TOON format

kpi_info = {
  "customer_lifetime_value": {
    "meaning": "Total premium revenue expected from a customer over their entire relationship with the insurer.",
    "math_formula": "SUM(premium_amount) WHERE policy.customer_id = customer.customer_id"
  },
  "premium_to_income_ratio": {
    "meaning": "Percentage of a customer’s income spent on insurance premiums. Indicates affordability.",
    "math_formula": "(premium_amount / annual_income) * 100"
  },
  "cross_sell_rate": {
    "meaning": "Percentage of customers who purchased more than one type of policy.",
    "math_formula": "(COUNT(customers_with_multiple_policies) / COUNT(total_customers)) * 100"
  },
  "lead_to_policy_conversion_rate": {
    "meaning": "Percentage of submissions that convert into active insurance policies.",
    "math_formula": "(COUNT(policies_created) / COUNT(total_submissions)) * 100"
  },
  "quote_conversion_rate": {
    "meaning": "Percentage of quotes that result in a bound policy.",
    "math_formula": "(COUNT(quotes_bound) / COUNT(total_quotes)) * 100"
  },
  "average_conversion_time": {
    "meaning": "Average number of days between submission and policy issuance.",
    "math_formula": "AVG(policy_start_date - submission_date)"
  },
  "agent_productivity": {
    "meaning": "Number of policies sold by each agent.",
    "math_formula": "COUNT(policies WHERE policies.agent_id = agents.agent_id)"
  },
  "agent_premium_contribution": {
    "meaning": "Total premium generated by an agent.",
    "math_formula": "SUM(premium_amount WHERE policy.agent_id = agent.agent_id)"
  },
  "submission_to_quote_time": {
    "meaning": "Average time taken to convert a submission to a quote.",
    "math_formula": "AVG(quote_date - submission_date)"
  },
  "claim_frequency": {
    "meaning": "Number of claims filed per 1000 policies.",
    "math_formula": "(COUNT(claims) / COUNT(policies)) * 1000"
  },
  "claim_severity": {
    "meaning": "Average financial size of claims filed.",
    "math_formula": "AVG(claim_amount)"
  },
  "loss_ratio": {
    "meaning": "Measures insurer profitability by comparing claim payouts to premium revenue.",
    "math_formula": "SUM(approved_amount) / SUM(premium_amount)"
  },
  "claim_processing_time": {
    "meaning": "Average number of days taken to resolve a claim.",
    "math_formula": "AVG(settlement_date - claim_date)"
  },
  "sla_compliance_rate": {
    "meaning": "Percentage of claims resolved within the defined service time.",
    "math_formula": "(COUNT(claims_within_SLA) / COUNT(total_claims)) * 100"
  },
  "high_loss_customers": {
    "meaning": "Customers whose claim payouts exceed premium contributions.",
    "math_formula": "SUM(approved_amount) > SUM(premium_amount)"
  },
  "policy_growth_rate": {
    "meaning": "Business growth based on increase in number of active policies.",
    "math_formula": "((policies_this_period - policies_last_period) / policies_last_period) * 100"
  },
  "premium_growth_rate": {
    "meaning": "Growth in premium revenue over a period.",
    "math_formula": "((total_premium_this_period - total_premium_last_period) / total_premium_last_period) * 100"
  },
  "customer_retention_rate": {
    "meaning": "Percentage of customers who renewed or kept policies active.",
    "math_formula": "(active_customers_current_period / active_customers_previous_period) * 100"
  },
  "expense_ratio": {
    "meaning": "Portion of earned premium consumed by operational expenses.",
    "math_formula": "operational_expenses / earned_premium"
  },
  "combined_ratio": {
    "meaning": "Core profitability metric combining loss and expense performance.",
    "math_formula": "loss_ratio + expense_ratio"
  },
  "customer_profitability": {
    "meaning": "Net financial value of a customer based on premium vs claim payout.",
    "math_formula": "SUM(premium_amount) - SUM(approved_amount)"
  },
  "risk_adjusted_premium_ratio": {
    "meaning": "Premium collected relative to risk level of a policy.",
    "math_formula": "premium_amount / policy_risk_level"
  },
  "claims_per_customer": {
    "meaning": "Average number of claims filed per customer.",
    "math_formula": "COUNT(claim_id) / COUNT(customer_id)"
  },
  "pending_claim_ratio": {
    "meaning": "Percentage of claims still under review or not yet settled.",
    "math_formula": "(COUNT(claims WHERE claim_status IN ['pending','review']) / COUNT(total_claims)) * 100"
  }
}

In [13]:
kpi_info_toon = ''
for i in kpi_info:
  kpi_info_toon += f'{i}\n'
  kpi_info_toon += encode(kpi_info[i])
  kpi_info_toon += '\n\n'

In [14]:
print(kpi_info_toon)

customer_lifetime_value
meaning: Total premium revenue expected from a customer over their entire relationship with the insurer.
math_formula: SUM(premium_amount) WHERE policy.customer_id = customer.customer_id

premium_to_income_ratio
meaning: Percentage of a customer’s income spent on insurance premiums. Indicates affordability.
math_formula: (premium_amount / annual_income) * 100

cross_sell_rate
meaning: Percentage of customers who purchased more than one type of policy.
math_formula: (COUNT(customers_with_multiple_policies) / COUNT(total_customers)) * 100

lead_to_policy_conversion_rate
meaning: Percentage of submissions that convert into active insurance policies.
math_formula: (COUNT(policies_created) / COUNT(total_submissions)) * 100

quote_conversion_rate
meaning: Percentage of quotes that result in a bound policy.
math_formula: (COUNT(quotes_bound) / COUNT(total_quotes)) * 100

average_conversion_time
meaning: Average number of days between submission and policy issuance.
mat

## Loading Models

In [15]:
def use_mistral(prompt, model="mistral-large-2411"):

  with Mistral(
    api_key=mistral_key,
) as mistral:

    res = mistral.chat.complete(model=model, messages=[
        {
            "content": prompt,
            "role": "user",
        },
    ], stream=False)

    return res.choices[0].message.content

In [16]:
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct", device_map='auto')
qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-7B-Instruct", device_map='auto')

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [17]:
rel_schema = {
  "type": "object",
  "properties": {
    "relevance": {
      "type": "string",
      "enum": ["Yes", "No"],
      "description": "Indicates whether the user query is relevant to the insurance domain knowledge, NL query understanding task, or concept defined by the system."
    },
    "explanation": {
      "type": "string",
      "description": "A brief explanation describing why the query was classified as relevant or not, focusing on intent, insurance context, and semantic alignment."
    },
    "kpi": {
      "type": "string",
      "description": "If a KPI is used give formula and information related to that KPI, else give None."
    }
  },
  "required": ["relevance", "explanation", "kpi"]
}

In [18]:
knowledge_schema = {
  "type": "object",
  "properties": {
    "Tables": {
      "type": "array",
      "items": {
        "type": "string",
        "description": "Name of a table that is required to answer the user's query."
      },
      "description": "List of tables that should be used in the SQL query."
    },
    "Columns": {
      "type": "array",
      "items": {
        "type": "string",
        "description": "Fully qualified column names that the SQL query must reference."
      },
      "description": "List of columns from the selected tables."
    },
    "Steps": {
      "type": "array",
      "items": {
        "type": "string",
        "description": "Step-by-step reasoning for building the SQL query."
      },
      "description": "Necessary steps to take to generate the SQL query."
    }
  },
  "required": ["Tables", "Columns", "Steps"]
}


## Setting Up DB

In [19]:
def colname_edit(text):
    text = text.lower().replace(" ", "_")
    return text

def update_db():
    conn = sqlite3.connect("insurancedata.db")

    data = ["agents.csv", "customers.csv", "policies.csv", "claims.csv", "submissions.csv"]

    for i in data:
        df = pd.read_csv(i)
        col_old = df.columns.tolist()
        column_new = [colname_edit(col) for col in col_old]
        df.rename(columns=dict(zip(col_old, column_new)), inplace=True)
        table_name = i.split(".")[0]
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    conn.commit()
    conn.close()
    print("Database updated successfully.")

In [21]:
update_db()

Database updated successfully.


In [22]:
conn = sqlite3.connect("insurancedata.db")
cursor = conn.cursor()
cursor.execute("SELECT * FROM agents;")
rows = cursor.fetchall()
print(rows)

conn.close()

[('A1000', 'Stephanie Taylor', 'LIC-E9361F05', 'Norman Inc', 'Telemarketing', 'auto'), ('A1001', 'Tracie Pena', 'LIC-54548090', 'Castaneda, Foster and Vang', 'Telemarketing', 'auto'), ('A1002', 'Allen Hall', 'LIC-36FFBEDC', 'Black Group', 'Digital', 'auto'), ('A1003', 'Michael Mcknight', 'LIC-750636F3', 'Simpson-Villa', 'Branch', 'auto'), ('A1004', 'Kelly Torres', 'LIC-EA29BA46', 'Anderson-Diaz', 'Telemarketing', 'home'), ('A1005', 'Jasmine Moore', 'LIC-7172AD8D', 'Jackson and Sons', 'Telemarketing', 'home'), ('A1006', 'Zachary Brown', 'LIC-90C66926', 'Morris-Burton', 'Digital', 'home'), ('A1007', 'Chelsea Weeks', 'LIC-AF82B387', 'Miller LLC', 'Branch', 'home'), ('A1008', 'David Lopez', 'LIC-ED8419CA', 'Stewart-Young', 'Telemarketing', 'health'), ('A1009', 'Denise Rojas', 'LIC-92C27DE0', 'Matthews-Hebert', 'Digital', 'health'), ('A1010', 'Carlos Thomas', 'LIC-360DDE52', 'Kelley-Smith', 'Telemarketing', 'life'), ('A1011', 'Anthony Floyd', 'LIC-8486C84A', 'Chapman-Perez', 'Digital', 'lif

In [23]:
def run_query(query):
  conn = sqlite3.connect("insurancedata.db")
  cursor = conn.cursor()
  cursor.execute(query)
  rows = cursor.fetchall()
  result = pd.DataFrame(rows, columns=[description[0] for description in cursor.description])

  conn.close()

  return result

## Code

In [24]:
use_mistral("Hello, how are you?")

"Hello! I'm just a computer program, so I don't have feelings, but thank you for asking. How can I assist you today?"

In [25]:
def use_qwen(prompt, schema, max_number_tokens, max_string_token_length,
            model=qwen_model, tokenizer=qwen_tokenizer):

  model.resize_token_embeddings(len(tokenizer))
  jsonformer = Jsonformer(
                model,
                tokenizer,
                schema,
                prompt,
                max_number_tokens=max_number_tokens,
                max_string_token_length=max_string_token_length,
            )
  output = jsonformer()

  return output

In [36]:
def prompts(prompt_type, args):
  nl_query = args['nl_query']

  if prompt_type == "knowledge":
    schema = args['schema']
    kpi_used = args['kpi_used']
    prompt = f"""You are a insurance domain specialist, who provides knowledge and steps for the natural language query given to you so that these steps can used to convert this NL query to SQL query.
Additionally, you will also be given the KPI information used in the NL query.
Give good and detailed knowledge in 300 words for the schema given below.
Your output should be in a JSON FORMAT and NO OTHER TEXT.

This knowledge SHOULD include:
- What tables to select
- What columns to use
- Steps to fetch and answer the query completely and correctly. If KPI was used, include steps to do that in SQL as well

Output format:
{{
  Tables: list of tables that should be used,
  Columns: List of columns from each table,
  Steps: Necessary steps to take to generate the SQL query, include how to approach the KPI formula if KPI was used.
}}

Schema:
{schema}

KPI Information:
{kpi_used}

Natural Language Query:
{nl_query}
"""

  if prompt_type == "relevance":
    kpi_info_args = args['kpi']
    prompt = f"""You are a insurance domain specialist, you'll be given a natural language query and KPIs related to the domain.
Your task is to check if this natural language query is relevant to insurance domain or not, so that if it is relevant the downstream task will be to convert this natural language query to SQL.
If the natural language query asks or uses any KPI, give that KPI information as well else give None.
You should also explain your choice.
Your output should be in JSON FORMAT and NO OTHER TEXT.

KPIs:
{kpi_info_args}

Output Format:
{{
  relevance: Yes/No,
  explanation: Explain why you chose yes or no.
  kpi: If a KPI is used give formula and information related to that KPI, else give None.
}}

Natural Language Query:
{nl_query}
"""

  if prompt_type == "sql":
    knowledge = args['knowledge']
    fetched_schema = args['req_schema']
    prompt = f"""You are a Insurance SQL Analyst, you'll be given a natural language query and steps to generate the SQL query.
Your task is to convert this natural language query to SQL.

Your output should be in a JSON format and NO OTHER TEXT.

Tables to use: {knowledge['Tables']}

Columns to use:
{encode(knowledge['Columns'])}

Steps to generate SQL query:
{knowledge['Steps']}


Schema of these tables:
{fetched_schema}

Output Format:
{{
  sql_query: Executable SQL query to answer the natural language query.
}}

Natural Language Query:
{nl_query}
"""
  if prompt_type == "reverse":
    sql_query = args['sql_query']
    prompt = f"""You are an Insurance domain SQL query analyst and guide.
You will be given an SQL query and your job is to clearly describe what that query is doing tying it to the insurance domain.

Your output should only be the explanation of the SQL query and NO OTHER TEXT.

SQL Query:
{sql_query}
    """

  if prompt_type == "sql_logical":
    sql_query = args['sql_query']
    fetched_schema = args['req_schema']
    knowledge = args['knowledge']
    reverse = args['reverse']
    prompt = f"""You are an Insurance domain SQL query analyst and guide.
You will be given a natural language query, its generated SQL query and the text description of that the SQL query is doing.
You'll also be given the tables and their schemas, and columns in SQL along with the steps used in generating SQL query that answer's the natural language query.

Use this information and check if the text description answers the natural language query or not, additionally also give the correct the SQL query if it is wrong.

Your output should be in a JSON format and NO OTHER TEXT.

Output Format:
{{
  correct: Yes/No,
  corrected_sql_query: If no, give the corrected SQL query to better answer the natural language query. If yes, None
}}


Tables to use: {knowledge['Tables']}

Columns to use:
{encode(knowledge['Columns'])}

Steps to generate SQL query:
{f"""{knowledge['Steps']}"""}

Schema of these tables:
{fetched_schema}

Natural Language Query:
{nl_query}

SQL Query:
{sql_query}

Explanation of SQL query as text:
{reverse}

"""

  if prompt_type == "sql_syntax":
    sql_query = args['sql_query']
    syn_sug = args['syn_sug']
    prompt = f"""You are an SQL query generator who reflects on their own mistakes and gives out the correct query.
You will be given the natural language query which was converted to SQL query in the previous iteration, the SQL query and the suggestions to correct the syntax error that occured previously.
Your job is to go through the given information and correct the SQL query syntactically.

Your output should be in a JSON format and NO OTHER TEXT.

Output Format:
{{
  corrected_sql_query: Executable SQL query to answer the natural language query.
}}

Natural Language Query:
{nl_query}

SQL Query:
{sql_query}

Suggestion to correct error occured previously:
{syn_sug}

    """

  if prompt_type == "syntax":
    error = args['syntax']
    sql_query = args['sql_query']
    fetched_schema = args['req_schema']
    knowledge = args['knowledge']
    prompt = f"""You are an evaluator and a guide for SQL query executions. You'll be given a natural language query, the SQL query for it that faced an error, tables and their schema, and columns that the SQL query is using to answer the natural language query.
Your job is to suggest revisions to the SQL query to ensure safe execution of the SQL query generated from the natural language query.

Tables to use: {knowledge['Tables']}

Columns to use:
{encode(knowledge['Columns'])}

Steps to generate SQL query:
{knowledge['Steps']}

Schema of these tables:
{fetched_schema}

Natural Language Query:
{nl_query}

SQL Query:
{sql_query}

Error:
{error}
"""

  return prompt

In [29]:
def get_args(nl_query, schema, kpi_info_toon):
  return {"nl_query": nl_query,
          "schema": schema,
          "kpi": kpi_info_toon}

In [50]:
# relevance_op = use_mistral(rel_prompt)

In [69]:
# print(relevance_op)

```json
{
  "relevance": "Yes"
}
```


In [70]:
# relevance_op = use_qwen(rel_prompt, rel_schema, 100, 100)

In [71]:
# relevance_op

{'relevance': 'No',
 'explanation': "The query is about the cost of a swimming pool which does not have any direct connection with insurance domain. It's more related to property valuation or construction cost."}

In [ ]:
# Catch the error and re-iterate
# Catch the logical error and re-iterate
# Reverse engineer
# Compare against ground truth and compare NL, reverese engineered SQL

In [55]:
nl_query = "What is the quote to bind ratio per agent channel? Use float numbers."
args = get_args(nl_query, metadata_toon, kpi_info_toon)
n_iters = 3

# Check Relevance
start = time.perf_counter()
rel_prompt = prompts("relevance", args)
relevance_op = use_mistral(rel_prompt)
relevance_op = eval(relevance_op.split("```")[1][4:])
# relevance_op = use_qwen(rel_prompt, rel_schema, 100, 100)
print("Relevance OP:")
print(relevance_op)


# Generating Knowledge that helps the generation
if relevance_op['relevance'] == 'No':
  print(relevance_op['explanation'])
else:
  args['kpi_used'] = relevance_op['kpi']
  knowledge_op = use_mistral(prompts("knowledge", args))
  print()
  print("Knowledge OP")
  print(knowledge_op)
  knowledge_op = eval(knowledge_op.split("```")[1][4:])
  args['knowledge'] = knowledge_op
  fetched_schema = ''
  for i in knowledge_op['Tables']:
    fetched_schema += f'{i}\n'
    fetched_schema += encode(new_metadata[i])
    fetched_schema += '\n\n'

  args['req_schema'] = fetched_schema

  # Generating SQL Query
  sql_prompt = prompts("sql", args)
  sql_op = use_mistral(sql_prompt)
  end = time.perf_counter()
  print(f"Time taken: {round(end - start, 2)} seconds")

  sql_query = eval(sql_op.split("```")[1][4:])['sql_query']

  print(f"NL Query: {nl_query}")
  print(f"SQL Query: {sql_query}")
  args['sql_query'] = sql_query

  # Executing Query
  for i in range(n_iters):
    print(f"Passing Syntax Error Trial: {i+1}")
    print("--------------------------------------------------------------")
    try:
      result = run_query(sql_query)
      print("No Syntax Error....")
      print("--------------------------------------------------")
      # print("Results Fetched:")
      # print(result)
      # print("--------------------------------------------------")
    except sqlite3.Error as error:
      # error = e
      print(f"Syntax Error: {error}")
      args['syntax'] = error
      print("Reflecting on Syntax Error....")
      syntax_prompt = prompts("syntax", args) # TODO: Direct query
      syntax_op = use_mistral(syntax_prompt)
      print()
      print("Syntax OP:")
      print(syntax_op)
      args['syn_sug'] = syntax_op
      sql_prompt = prompts("sql_syntax", args)
      sql_op = use_mistral(sql_prompt)
      print()
      print("SQL OP:")
      print(sql_op)
      sql_query = eval(sql_op.split("```")[1][4:])['corrected_sql_query']
      args['sql_query'] = sql_query
      continue

    # Checking for logical errors
    print(f"Passing Logical Error Trial: {i+1}")
    print("--------------------------------------------------------------")
    print("Reverse Engineering")
    reverse_prompt = prompts("reverse", args)
    reversed = use_mistral(reverse_prompt)
    print()
    print("Reverse OP:")
    print(reversed)
    args['reverse'] = reversed
    sql_prompt = prompts("sql_logical", args)
    sql_op = use_mistral(sql_prompt)
    sample = eval(sql_op.split("```")[1][4:])
    sql_query_sample = sample['corrected_sql_query']
    # args['sql_query'] = sql_query


    if sample['correct'] == 'Yes':
      print("No Logical Error....")
      print("--------------------------------------------------")
      print("Results Fetched:")
      result = run_query(sql_query)
      print(result)
      print("--------------------------------------------------")
      break
    else:
      print("Logical Error....Retrying")
      # sql_op = use_mistral(sql_prompt)
      # print()
      # print("SQL op")
      # print(sql_op)
      sql_query = sql_query_sample
      args['sql_query'] = sql_query
      continue


  #   print("")

  # print("Checking for Syntax Errors...")
  # for i in range(n_iters):
  #   print(f"Passing Syntax Error Trial: {i+1}")
  #   print("--------------------------------------------------------------")
  #   try:
  #     result = run_query(sql_query)
  #     print("No Syntax Error....")
  #     print("--------------------------------------------------")
  #     # print("Results Fetched:")
  #     # print(result)
  #     # print("--------------------------------------------------")
  #     break
  #   except sqlite3.Error as error:
  #     # error = e
  #     print(f"Syntax Error: {error}")
  #     args['syntax'] = error
  #     print("Reflecting on Syntax Error....")
  #     syntax_prompt = prompts("syntax", args) # TODO: Direct query
  #     syntax_op = use_mistral(syntax_prompt)
  #     print()
  #     print("Syntax OP:")
  #     print(syntax_op)
  #     args['syn_sug'] = syntax_op
  #     sql_prompt = prompts("sql_syntax", args)
  #     sql_op = use_mistral(sql_prompt)
  #     print()
  #     print("SQL OP:")
  #     print(sql_op)
  #     sql_query = eval(sql_op.split("```")[1][4:])['corrected_sql_query']
  #     args['sql_query'] = sql_query
  #     continue
  # else:
  #   print(f"Tried {n_iters} times and Failed..!")

  # # Checking for logical errors
  # for i in range(n_iters):
  #   print(f"Passing Logical Error Trial: {i+1}")
  #   print("--------------------------------------------------------------")
  #   print("Reverse Engineering")
  #   reverse_prompt = prompts("reverse", args)
  #   reversed = use_mistral(reverse_prompt)
  #   print()
  #   print("Reverse OP:")
  #   print(reversed)
  #   args['reverse'] = reversed
  #   sql_prompt = prompts("sql_logical", args)
  #   sql_op = use_mistral(sql_prompt)
  #   sample = eval(sql_op.split("```")[1][4:])
  #   sql_query_sample = sample['corrected_sql_query']
  #   # args['sql_query'] = sql_query


  #   if sample['correct'] == 'Yes':
  #     print("No Logical Error....")
  #     print("--------------------------------------------------")
  #     print("Results Fetched:")
  #     result = run_query(sql_query)
  #     print(result)
  #     print("--------------------------------------------------")
  #     break
  #   else:
  #     print("Logical Error....Retrying")
  #     # sql_op = use_mistral(sql_prompt)
  #     # print()
  #     # print("SQL op")
  #     # print(sql_op)
  #     sql_query = sql_query_sample
  #     args['sql_query'] = sql_query
  #     continue



Relevance OP:
{'relevance': 'Yes', 'explanation': 'The query is asking for a conversion rate which is a common KPI in the insurance domain. Specifically, it is requesting the quote to bind ratio per agent channel, which can be calculated using the quote_conversion_rate KPI.', 'kpi': 'quote_conversion_rate: (COUNT(quotes_bound) / COUNT(total_quotes)) * 100'}

Knowledge OP
```json
{
  "Tables": ["submissions", "agents"],
  "Columns": {
    "submissions": [
      "submission_id",
      "policy_type",
      "statusid",
      "agent_id"
    ],
    "agents": [
      "agent_id",
      "channel"
    ]
  },
  "Steps": [
    "Join the 'submissions' table with the 'agents' table on the 'agent_id' column.",
    "Filter the submissions to include only those with 'statusid' values of 'BND' (Bound) and 'QTE' (Quote Only).",
    "Count the number of quotes that were bound (statusid = 'BND') and the total number of quotes (statusid in ['BND', 'QTE']).",
    "Group the results by the 'channel' column fr

In [49]:
new_query = "SELECT agents.agent_id, agents.agent_name, agents.channel, (COUNT(CASE WHEN submissions.statusid = 'BND' THEN 1 END) * 1.0 / COUNT(CASE WHEN submissions.statusid = 'QTE' THEN 1 END)) * 100 AS quote_to_bind_ratio FROM submissions JOIN agents ON submissions.agent_id = agents.agent_id GROUP BY agents.agent_id, agents.agent_name, agents.channel"

In [50]:
run_query(new_query)

,agent_id,agent_name,channel,quote_to_bind_ratio
0,A1000,Stephanie Taylor,Telemarketing,35.897436
1,A1001,Tracie Pena,Telemarketing,31.869919
2,A1002,Allen Hall,Digital,39.838710
3,A1003,Michael Mcknight,Branch,32.223903
4,A1004,Kelly Torres,Telemarketing,26.879271
5,A1005,Jasmine Moore,Telemarketing,40.512821
6,A1006,Zachary Brown,Digital,31.853786
7,A1007,Chelsea Weeks,Branch,41.873278
8,A1008,David Lopez,Telemarketing,17.916667
9,A1009,Denise Rojas,Digital,17.555556


In [ ]:
nl_query = "What is the quote to bind ratio per agent channel? Use float numbers."
args = get_args(nl_query, metadata_toon, kpi_info_toon)
n_iters = 3

start = time.perf_counter()
rel_prompt = prompts("relevance", args)
relevance_op = use_mistral(rel_prompt)
relevance_op = eval(relevance_op.split("```")[1][4:])
print("Relevance OP:")
print(relevance_op)

if relevance_op['relevance'] == 'No':
    print(relevance_op['explanation'])

args['kpi_used'] = relevance_op['kpi']

# ---------- Knowledge ----------
knowledge_raw = use_mistral(prompts("knowledge", args))
print("\nKnowledge OP")
print(knowledge_raw)
knowledge_op = eval(knowledge_raw.split("```")[1][4:])
args['knowledge'] = knowledge_op

# Build required schema
schema = ""
for table in knowledge_op["Tables"]:
    schema += f"{table}\n"
    schema += encode(new_metadata[table]) + "\n\n"

args["req_schema"] = schema

# ---------- Initial SQL Generation ----------
sql_prompt = prompts("sql", args)
sql_op = use_mistral(sql_prompt)
sql_query = eval(sql_op.split("```")[1][4:])["sql_query"]
args["sql_query"] = sql_query

print(f"NL Query: {nl_query}")
print(f"Initial SQL Query: {sql_query}")

# =========================================================
# 🔥 Unified Syntax + Logical Correction Loop
# =========================================================
for iteration in range(n_iters):
    print(f"\n================ ITERATION {iteration+1} / {n_iters} ================")

    # ----------------------------------------
    # 1) TRY SYNTAX EXECUTION
    # ----------------------------------------
    print("Checking Syntax...")
    syntax_ok = False
    try:
        run_query(sql_query)
        syntax_ok = True
        print("Syntax OK ✓")
    except sqlite3.Error as err:
        print(f"❌ Syntax Error: {err}")
        args["syntax"] = err

        # Fix syntax
        syntax_prompt = prompts("syntax", args)
        syntax_op = use_mistral(syntax_prompt)
        args["syn_sug"] = syntax_op

        sql_prompt = prompts("sql_syntax", args)
        sql_corrected = use_mistral(sql_prompt)
        sql_query = eval(sql_corrected.split("```")[1][4:])["corrected_sql_query"]

        args["sql_query"] = sql_query
        print("→ Retrying next iteration...")
        continue  # Jump to next iteration

    # ----------------------------------------
    # 2) IF SYNTAX OK → CHECK LOGICAL
    # ----------------------------------------
    print("Checking Logical Correctness...")
    reverse_prompt = prompts("reverse", args)
    reverse_op = use_mistral(reverse_prompt)
    args["reverse"] = reverse_op

    logical_prompt = prompts("sql_logical", args)
    logical_analysis = use_mistral(logical_prompt)
    logical_output = eval(logical_analysis.split("```")[1][4:])

    if logical_output["correct"] == "Yes":
        print("Logical Check OK ✓")
        print("Final Results:")
        print(run_query(sql_query))
        break
    else:
        print("❌ Logical Error. Fixing...")
        sql_query = logical_output["corrected_sql_query"]
        args["sql_query"] = sql_query
        continue

else:
    print("\n❌ Failed after all iterations.")
